In [215]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm
import re

In [5]:
import re
p = re.compile('ab*')
p # re.compile(r'ab*', re.UNICODE)

re.compile(r'ab*', re.UNICODE)

In [6]:
re.UNICODE.value

32

In [29]:
p = re.compile('[a-t]+')
m = p.match("temupau")
print(m) # <re.Match object; span=(0, 3), match='tem'>

<re.Match object; span=(0, 3), match='tem'>


In [32]:
m.group() # 'tem'
m.start() # 0
m.end()   # 3
m.span()  # (0, 3)

(0, 3)

In [39]:
print(p.match('::: message')) # None
m = p.search('::: message'); print(m) #<re.Match object; span=(4, 11), match='message'>
m.group() #'message'
m.start() # 4
m.end() # 11
m.span() # (4, 11)

None
<re.Match object; span=(4, 11), match='message'>


(4, 11)

In [44]:
p = re.compile(r'[a-r]+')
m = p.match('ramesh tau')
if m:
    print('Matched', m.group())
else:
    print('No Match')
## Output - Matched rame

Matched rame


In [45]:
p = re.compile(r'\d+')
p.findall('12 drummers drumming, 11 pipers piping, 10 lords a-leaping')

['12', '11', '10']

In [49]:
p = re.compile(r'\d+')
iterator = p.finditer('12 drummers drumming, 11 pipers piping, 10 lords a-leaping')
for match in iterator:
    print(match.group())
    print(match.start())
    print(match.end())
    print(match.span())

12
0
2
(0, 2)
11
22
24
(22, 24)
10
40
42
(40, 42)


In [50]:
p = re.compile(r'[a-m]')
p.search('allubro')

<re.Match object; span=(0, 1), match='a'>

In [4]:
print(re.match(r'From\s+', 'Fromage amk')) # None
print(re.match(r'From\s+', 'From 7 till 10')) # <re.Match object; span=(0, 5), match='From '>

None
<re.Match object; span=(0, 5), match='From '>


In [6]:
print(re.search(r'^From', 'From here to eternity')) # <re.Match object; span=(0, 4), match='From'>
print(re.search(r'^From', 'Reciting From Memory')) # None

<re.Match object; span=(0, 4), match='From'>
None


In [11]:
p = re.compile('\bclass\b')
print(p.search('This is my class')) # None
print(p.search('THis is declassified')) # None
print(p.search('One subclass at a time')) # None
print(p.search('\b' + 'class' + '\b')) # <re.Match object; span=(0, 7), match='\x08class\x08'>

None
None
None
<re.Match object; span=(0, 7), match='\x08class\x08'>


In [17]:
p = re.compile('(ab)*')
print(p.match('ababbbababb').span(1)) # (2, 4)

(2, 4)


In [23]:
p = re.compile('(a)b')
print(p.match('ab')) # <re.Match object; span=(0, 2), match='ab'>
m = p.match('ab')
print(m.group(0)) # ab
print(m.group(1)) # a

<re.Match object; span=(0, 2), match='ab'>
ab
a


In [30]:
p = re.compile('(a(b)c)d')
m = p.match('abcd')
m.group() # 'abcd'
m.group(1) # 'abc'
m.group(2) # b
m.group(2,1,2) # ('b', 'abc', 'b')
m.groups() # ('abc', 'b')

('abc', 'b')

In [38]:
p = re.compile(r'\b(\w+)\s+\1\b')
p.search('This is the the Paris').group() # 'the the'

'the the'

In [40]:
m = re.match("([abc])+", "abc")
m.groups() # ('c',)
m = re.match("(?:[abc])+", "abc")
m.groups() # ()

()

In [45]:
p = re.compile(r'\W+')
p.split('This is a test, short and sweet, of split().')
p.split('This is a test, short and sweet, of split().', 3)


['This', 'is', 'a', 'test, short and sweet, of split().']

In [46]:
p = re.compile(r'\W+')
p2 = re.compile(r'(\W+)')

p.split('This.... is a test.') # ['This', 'is', 'a', 'test', '']
p2.split('This.... is a test.') # ['This', '.... ', 'is', ' ', 'a', ' ', 'test', '.', '']

['This', '.... ', 'is', ' ', 'a', ' ', 'test', '.', '']

In [48]:
p = re.compile('(blue|white|red)', re.IGNORECASE)
p.sub('color', 'blue shocks and Red shoes', count=1) # 'color shocks and Red shoes'

'color shocks and Red shoes'

In [53]:
p = re.compile('(blue|white|red)', re.IGNORECASE)
p.subn('color', 'blue and red')
p.subn('color', 'blue and red', count=1)

('color and red', 1)

In [55]:
p = re.compile(r'x*')
p.sub('-', 'abxd cx')

'-a-b--d- -c--'

In [57]:
p = re.compile('section{ ( [^}]* ) }', re.VERBOSE)
p.sub(r'subsection{\1}','section{First} section{second}')

'subsection{First} subsection{second}'

In [111]:
df = DataFrame({
    'Name':['A','A','A','A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C'],
    'Year':np.tile([1980, 1981, 1982, 1983], 3),
    'Col1':[np.nan, np.nan, 22.74, 15.50, np.nan, 13.5, 17., 19.3, 12., 15.5, 16.36, 22.74],
    'Col3':[9817, 11487, 12256, 14214, np.nan, np.nan, np.nan, np.nan, 1750, 1790, 1790, 1810]
})
df

,Name,Year,Col1,Col3
0,A,1980,NaN,9817.0
1,A,1981,NaN,11487.0
2,A,1982,22.74,12256.0
3,A,1983,15.50,14214.0
4,B,1980,NaN,NaN
5,B,1981,13.50,NaN
6,B,1982,17.00,NaN
7,B,1983,19.30,NaN
8,C,1980,12.00,1750.0
9,C,1981,15.50,1790.0


In [136]:
import pandas as pd

d = {0: {50: 16, 23: 7, 22: 7, 5: 1, 31: 10, 47: 15, 2: 0, 44: 14, 39: 13, 21: 7, 11: 3, 30: 10, 16: 5, 42: 14, 26: 8, 1: 0, 40: 13, 32: 10, 34: 11, 46: 15, 49: 16, 15: 5, 37: 12, 9: 3, 12: 4, 41: 13, 24: 8, 10: 3, 18: 6, 8: 2, 33: 11, 35: 11, 0: 0, 36: 12, 27: 9, 48: 16, 38: 12, 7: 2, 28: 9, 3: 1, 29: 9, 43: 14, 4: 1, 45: 15, 17: 5, 14: 4, 13: 4, 19: 6, 25: 8, 20: 6, 6: 2}, 1: {50: 2, 23: 2, 22: 1, 5: 2, 31: 1, 47: 2, 2: 2, 44: 2, 39: 0, 21: 0, 11: 2, 30: 0, 16: 1, 42: 0, 26: 2, 1: 1, 40: 1, 32: 2, 34: 1, 46: 1, 49: 1, 15: 0, 37: 1, 9: 0, 12: 0, 41: 2, 24: 0, 10: 1, 18: 0, 8: 2, 33: 0, 35: 2, 0: 0, 36: 0, 27: 0, 48: 0, 38: 2, 7: 1, 28: 1, 3: 0, 29: 2, 43: 1, 4: 1, 45: 0, 17: 2, 14: 2, 13: 1, 19: 1, 25: 1, 20: 2, 6: 0}, 2: {50: 32, 23: 32, 22: 32, 5: 32, 31: 32, 47: 32, 2: 32, 44: 33, 39: 46, 21: 64, 11: 97, 30: 97, 16: 97, 42: 99, 26: 99, 1: 99, 40: 99, 32: 100, 34: 100, 46: 100, 49: 101, 15: 101, 37: 101, 9: 101, 12: 102, 41: 103, 24: 103, 10: 103, 18: 103, 8: 104, 33: 105, 35: 105, 0: 106, 36: 108, 27: 109, 48: 109, 38: 110, 7: 110, 28: 111, 3: 111, 29: 111, 43: 111, 4: 111, 45: 111, 17: 112, 14: 112, 13: 114, 19: 116, 25: 116, 20: 121, 6: 122}}
#print (pd.DataFrame(d).pivot(0,1,2).applymap(chr).agg(''.join))
pd.DataFrame(d).pivot(0,1,2).applymap(chr).agg(''.join)

1
0    jozefeg@gmail.com
1    congrat to decode
2      happy coding!  
dtype: object

In [116]:
df.groupby(['Name'], as_index=False).last()

,Name,Year,Col1,Col3
0,A,1983,15.50,14214.0
1,B,1983,19.30,NaN
2,C,1983,22.74,1810.0


In [94]:
for i in zip(*[iter(list_arr)]*4):
    print(np.vstack(i))

[[6300 6675    3]
 [5560 6675    3]
 [5560 6675    3]
 [5560 6675    3]]
[[6300 6675    3]
 [5560 6675    3]
 [5560 6675    3]
 [5560 6675    3]]


In [66]:
for i in l[0:len(l):4]:
    print(np.vstack(i))

[[6300]
 [6675]
 [   3]]
[[6300]
 [6675]
 [   3]]


In [76]:
np.vstack((list_arr[0], list_arr[1], list_arr[2], list_arr[3]))

array([[6300, 6675,    3],
       [5560, 6675,    3],
       [5560, 6675,    3],
       [5560, 6675,    3]])

In [104]:
for i in np.vsplit(list_arr, len(list_arr)//4):
    print(i)

[[6300 6675    3]
 [5560 6675    3]
 [5560 6675    3]
 [5560 6675    3]]
[[6300 6675    3]
 [5560 6675    3]
 [5560 6675    3]
 [5560 6675    3]]


In [207]:
DS_df = DataFrame({
    'LAultimateparentcountry':['India', 'Germany', 'India'],
    'borrowerultimateparentcountry':['France', 'Ireland', 'France'],
    'tot':[56708, 87902, 91211]
})
for i in range(23):
    DS_df = pd.concat([DS_df, DS_df])
DS_df.shape

(6291456, 3)

In [208]:
%timeit pd.Series(DS_df.tot.values, index=DS_df.LAultimateparentcountry.str.cat(DS_df.borrowerultimateparentcountry, sep="_")).to_dict()

3.92 s ± 77.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [209]:
%timeit dict(zip(DS_df.LAultimateparentcountry.str.cat(DS_df.borrowerultimateparentcountry, sep="_"), DS_df.tot))

3.94 s ± 226 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [190]:
from collections import defaultdict
dict_map = defaultdict(list)

for index, row in DS_df.iterrows():
    dict_map[index].append([row["LAultimateparentcountry"],
                        row["borrowerultimateparentcountry"],
                        row["tot"]])

In [191]:
def test():
    countries_map = {}
    for index, row in DS_df.iterrows():
        if row["tot"] not in countries_map.values():
            countries_map[f'{row["LAultimateparentcountry"]}_{row["borrowerultimateparentcountry"]}'] = row["tot"]
    return countries_map

In [203]:
%timeit test()

583 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [211]:
data = {'Unnamed: 0': {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7, 7: 8, 8: 9, 9: 10, 10: 11, 11: 12, 12: 13, 13: 14, 14: 15, 15: 16, 16: 17, 17: 18, 18: 19, 19: 20, 20: 21, 21: 22, 22: 23, 23: 24, 24: 25, 25: 26, 26: 27, 27: 28}, 
        'id': {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 2, 13: 2, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 2, 21: 2, 22: 2, 23: 2, 24: 2, 25: 2, 26: 2, 27: 2}, 
        'activity': {0: 'start', 1: 'a', 2: 'b', 3: 'c', 4: 'c', 5: 'd', 6: 'e', 7: 'stop', 8: 'start', 9: 'b', 10: 'stop', 11: 'f', 12: 'stop', 13: 'start', 14: 'a', 15: 'a', 16: 'b', 17: 'stop', 18: 'start', 19: 'f', 20: 'g', 21: 'h', 22: 'h', 23: 'stop', 24: 'start', 25: 'e', 26: 'f', 27: 'stop'}, 
        'duration_sec': {0: 0.7, 1: 0.3, 2: 0.4, 3: 0.5, 4: 0.5, 5: 0.4, 6: 0.6, 7: 2.0, 8: 0.1, 9: 0.3, 10: 0.2, 11: 0.3, 12: 40.0, 13: 3.0, 14: 0.7, 15: 3.0, 16: 0.2, 17: 0.2, 18: 0.1, 19: 0.4, 20: 0.2, 21: 0.5, 22: 6.0, 23: 9.0, 24: 0.2, 25: 0.3, 26: 0.4, 27: 0.7}, 'cycle': {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 2, 9: 2, 10: 2, 11: 3, 12: 4, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 18: 6, 19: 6, 20: 6, 21: 6, 22: 6, 23: 6, 24: 7, 25: 7, 26: 7, 27: 7}, 'intended_cycle': {0: 1, 1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 3, 7: 3, 8: 4, 9: 4, 10: 4, 11: 5, 12: 6, 13: 7, 14: 7, 15: 7, 16: 7, 17: 7, 18: 8, 19: 8, 20: 8, 21: 8, 22: 8, 23: 8, 24: 9, 25: 9, 26: 10, 27: 10}}

df = pd.DataFrame.from_dict(data)
df

,Unnamed: 0,id,activity,duration_sec,cycle,intended_cycle
0,1,1,start,0.7,1,1
1,2,1,a,0.3,1,1
2,3,1,b,0.4,1,1
3,4,1,c,0.5,1,1
4,5,1,c,0.5,1,2
5,6,1,d,0.4,1,2
6,7,1,e,0.6,1,3
7,8,1,stop,2.0,1,3
8,9,1,start,0.1,2,4
9,10,1,b,0.3,2,4


In [213]:
AntCris=Datos.loc['2005-01-01':'2007-07-01']
for x in AntCris.columns:
  aux=AntCris[x].dropna()
  aux2=aux.iloc[0]
  print(x)
  for y in AntCris[x].values:
    if np.isnan(y)==True:
      b=2
    else:
      a=np.log(y)-np.log(aux2)
      AntCris[x]=AntCris[x].replace([y],a)

NameError: name 'Datos' is not defined

In [218]:
from io import StringIO
import csv
data = """label1,label2,label3
index1,"a,c,e
index2,b,d,f"""

dia = csv.excel()
dia.quoting = csv.QUOTE_NONE
pd.read_csv(StringIO(data), dialect=dia)


,label1,label2,label3
index1,"""a",c,e
index2,b,d,f


In [219]:
Toy_diz = {'X180213_1_0004_1_000001': {'2-3': 46.75512255828233, '7-9': 32.30237861210842, '10-11': 28.538901663518867, '0-1': 35.517087709439245, '0-2': 51.17852562784512,
                              '2-5': 55.15972203555779, '3-5': 26.5812809322651, '11-12': 24.76573298410529, '5-7': 32.37719839640236, '8-10': 42.19915601810067,
                               '3-4': 23.072183092199992, '7-8': 22.892458496194777, '5-6': 28.625171066039066, '8-9': 27.6671947258843, '6-8': 32.379920582978585,
                            '4-6': 29.7290251276425,'6-7': 27.83287476636216, '9-11': 46.237038548765206, '9-10': 18.723337839178146, '4-5': 26.464402246036077}}
Toy_diz

{'X180213_1_0004_1_000001': {'2-3': 46.75512255828233,
  '7-9': 32.30237861210842,
  '10-11': 28.538901663518867,
  '0-1': 35.517087709439245,
  '0-2': 51.17852562784512,
  '2-5': 55.15972203555779,
  '3-5': 26.5812809322651,
  '11-12': 24.76573298410529,
  '5-7': 32.37719839640236,
  '8-10': 42.19915601810067,
  '3-4': 23.072183092199992,
  '7-8': 22.892458496194777,
  '5-6': 28.625171066039066,
  '8-9': 27.6671947258843,
  '6-8': 32.379920582978585,
  '4-6': 29.7290251276425,
  '6-7': 27.83287476636216,
  '9-11': 46.237038548765206,
  '9-10': 18.723337839178146,
  '4-5': 26.464402246036077}}

In [221]:
np.array(Toy_diz)

array({'X180213_1_0004_1_000001': {'2-3': 46.75512255828233, '7-9': 32.30237861210842, '10-11': 28.538901663518867, '0-1': 35.517087709439245, '0-2': 51.17852562784512, '2-5': 55.15972203555779, '3-5': 26.5812809322651, '11-12': 24.76573298410529, '5-7': 32.37719839640236, '8-10': 42.19915601810067, '3-4': 23.072183092199992, '7-8': 22.892458496194777, '5-6': 28.625171066039066, '8-9': 27.6671947258843, '6-8': 32.379920582978585, '4-6': 29.7290251276425, '6-7': 27.83287476636216, '9-11': 46.237038548765206, '9-10': 18.723337839178146, '4-5': 26.464402246036077}},
      dtype=object)